In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
import urllib.parse
import re

seoul_districts = [
    "서울 강남구 맛집", "서울 강동구 맛집", "서울 강북구 맛집", "서울 강서구 맛집", "서울 관악구 맛집", "서울 광진구 맛집", "서울 구로구 맛집", "서울 금천구 맛집", "서울 노원구 맛집", "서울 도봉구 맛집",
    "서울 동대문구 맛집", "서울 동작구 맛집", "서울 마포구 맛집", "서울 서대문구 맛집", "서울 서초구 맛집", "서울 성동구 맛집", "서울 성북구 맛집", "서울 송파구 맛집", "서울 양천구 맛집",
    "서울 영등포구 맛집", "서울 용산구 맛집", "서울 은평구 맛집", "서울 종로구 맛집", "서울 중구 맛집", "서울 중랑구 맛집"
]

def switch_left():
    driver.switch_to.default_content()
    iframe = driver.find_element(By.XPATH, '//*[@id="searchIframe"]')
    driver.switch_to.frame(iframe)

def switch_right():
    driver.switch_to.default_content()
    iframe = driver.find_element(By.XPATH, '//*[@id="entryIframe"]')
    driver.switch_to.frame(iframe)

options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(3)

for district in seoul_districts:
    search_url = f"https://map.naver.com/p/search/{urllib.parse.quote(district)}"
    driver.get(search_url)
    sleep(3)

    switch_left()

    try:
        next_button = driver.find_element(By.XPATH, '//a[@class="eUTV2" and .//span[text()="다음페이지"]]')
        next_page_disabled = next_button.get_attribute('aria-disabled')
    except:
        print(f"[{district}] 더 이상 페이지가 없습니다.")

    scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

    while True:
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)
        sleep(1)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
        if new_height == last_height:
            break
        last_height = new_height

    page_no = driver.find_element(By.XPATH, '//a[contains(@class, "mBN2s qxokY")]').text
    elements = driver.find_elements(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]//li')[2:] if page_no == '1' else driver.find_elements(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]//li')

    for index, element in enumerate(elements):
        try:
            switch_left()
            element.find_element(By.CLASS_NAME, 'CHC5F').find_element(By.XPATH, ".//a/div/div/span").click()
            sleep(2)
        except:
            print(f'{index+1}번째 가게 클릭 실패')
            continue

        try:
            switch_right()
            sleep(2)

            soup = BeautifulSoup(driver.page_source, "html.parser")

            title = soup.find("div", class_="LylZZ v8v5j")
            store_name = title.find("span", class_="GHAhO").get_text() if title else "가게명 없음"
            category = title.find("span", class_="lnJFt").get_text() if title else "카테고리 없음"

            # '카페,디저트' 카테고리는 스킵
            if category == "카페,디저트":
                continue

            rating_element = soup.find("span", class_="PXMot LXIwF")
            rating = rating_element.get_text(strip=True).replace("별점", "") if rating_element else "별점 없음"

            visitor_review_element = soup.find("a", string=lambda text: text and "방문자 리뷰" in text)
            visitor_review = visitor_review_element.get_text(strip=True) if visitor_review_element else "방문자 리뷰 없음"

            blog_review_element = soup.find("a", string=lambda text: text and "블로그 리뷰" in text)
            blog_review = blog_review_element.get_text(strip=True) if blog_review_element else "블로그 리뷰 없음"

            address_element = soup.find("span", class_="LDgIH")
            address = address_element.get_text() if address_element else "주소 없음"

            menu_list = []
            menu_prices = []  
            try:
                menu_tab = driver.find_element(By.XPATH, '//a[.//span[text()="메뉴"]]')
                menu_tab.click()
                sleep(2)

                soup = BeautifulSoup(driver.page_source, "html.parser")

                is_naver_order = bool(soup.find("ul", class_="order_list_area"))

                representative_menus = []
                all_menus = []

                if is_naver_order:
                    menu_items = soup.find_all("li", class_="order_list_item")

                    for item in menu_items:
                        name = item.find("div", class_="tit").text.strip() if item.find("div", class_="tit") else "메뉴 이름 없음"
                        price_element = item.find("div", class_="price").find("strong") if item.find("div", class_="price") else None
                        price = int(price_element.text.replace(",", "")) if price_element else 0
                        all_menus.append({"name": name, "price": f"{price:,}원", "price_value": price})

                        tag = item.find("span", class_="menu_tag popular")
                        if tag and "인기" in tag.text:
                            representative_menus.append({"name": name, "price": f"{price:,}원", "price_value": price})

                else:
                    menu_items = soup.find_all("li", class_="E2jtL")

                    for item in menu_items:
                        name = item.find("span", class_="lPzHi").text.strip() if item.find("span", class_="lPzHi") else "메뉴 이름 없음"
                        price_element = item.find("div", class_="GXS1X")
                        price = int(price_element.text.replace(",", "").replace("원", "").strip()) if price_element and re.search(r"\d+", price_element.text) else 0
                        all_menus.append({"name": name, "price": f"{price:,}원", "price_value": price})

                        tag = item.find("span", class_="QM_zp")
                        if tag and "대표" in tag.text:
                            representative_menus.append({"name": name, "price": f"{price:,}원", "price_value": price})

                menu_list = [m for m in representative_menus if m["price_value"] > 0] if representative_menus else sorted([m for m in all_menus if m["price_value"] > 0], key=lambda x: x["price_value"], reverse=True)[:3]

                avg_price = f"{sum(m['price_value'] for m in menu_list) // len(menu_list):,}원" if menu_list else "가격 정보 없음"

            except:
                print('메뉴 탭 오류')

            print(f'[{district.split()[1]}] {index+1}. {store_name} · {category}')
            print(f'평점: {rating} · {visitor_review} · {blog_review}')
            print(f'가게 주소: {address}')
            print(f'평균 가격: {avg_price}')
            print('메뉴 목록:')
            for menu in menu_list:
                print(f"- {menu['name']} : {menu['price']}")
            print('-' * 50)

        except Exception as ex:
            print(f"{index+1}번째 가게 정보 수집 실패: {ex}")
            print('-' * 50)
            continue


[강남구] 1. 어거스트 힐 강남점 · 양식
평점: 4.56 · 방문자 리뷰 6,634 · 블로그 리뷰 4,923
가게 주소: 서울 강남구 강남대로106길 25
평균 가격: 119,000원
메뉴 목록:
- 티본스테이크 (700g) : 119,000원
--------------------------------------------------
[강남구] 2. 갓포준 · 일식당
평점: 4.71 · 방문자 리뷰 330 · 블로그 리뷰 559
가게 주소: 서울 강남구 논현로163길 10 B1
평균 가격: 47,000원
메뉴 목록:
- 사시미 모리아와세 : 80,000원
- 갓포준 사합 : 98,000원
- 마구로 타르타르 : 35,000원
- 아보카도 마구로 사라다 : 28,000원
- 트러플 에비 크림 코로케 : 28,000원
- 토마토 나베 : 40,000원
- 후토마끼 4피스 : 20,000원
--------------------------------------------------
[강남구] 3. 츄라우미 역삼본점 · 요리주점
평점: 4.44 · 방문자 리뷰 1,745 · 블로그 리뷰 5,716
가게 주소: 서울 강남구 언주로93길 21 남전타운
평균 가격: 69,400원
메뉴 목록:
- 세이로무시 : 65,000원
- 모듬사시미 : 79,000원
- 특사시미 : 140,000원
- 매콤해산물토마토나베 : 33,000원
- 모듬튀김 : 30,000원
--------------------------------------------------
[강남구] 4. 이화옥 삼성본점 · 육류,고기요리
평점: 4.66 · 방문자 리뷰 702 · 블로그 리뷰 1,208
가게 주소: 서울 강남구 테헤란로83길 40 1층
평균 가격: 106,400원
메뉴 목록:
- 한우 스페셜 : 120,000원
- 돈모듬 : 60,000원
- 한우 황제 스페셜 : 270,000원
- 새우살 : 67,000원
- 오돌갈비 : 15,000원
--------------------------------

InvalidSessionIdException: Message: invalid session id
Stacktrace:
#0 0x58611cee4ffa <unknown>
#1 0x58611c9a37c3 <unknown>
#2 0x58611c9e65df <unknown>
#3 0x58611ca1b196 <unknown>
#4 0x58611ca15813 <unknown>
#5 0x58611ca14ae5 <unknown>
#6 0x58611c96c528 <unknown>
#7 0x58611ceac42b <unknown>
#8 0x58611ceb02ec <unknown>
#9 0x58611ce93a22 <unknown>
#10 0x58611ceb0e64 <unknown>
#11 0x58611ce77bef <unknown>
#12 0x58611c96af5b <unknown>
#13 0x73f71ba29d90 <unknown>


In [4]:
driver.quit()